# 1. <a id='toc1_'></a>[**NLP and Speech Recognition Chatbot**](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- 1. [**NLP and Speech Recognition Chatbot**](#toc1_)    
  - 1.1. [**Problem Statement**](#toc1_1_)    
  - 1.2. [**Objectives**](#toc1_2_)    
  - 1.3. [**Analysis To Be Done**](#toc1_3_)    
    - 1.3.1. [**Ensure Necessary Modules are Installed**](#toc1_3_1_)    
    - 1.3.2. [**Import Modules and Set Default Environment Variables and Load Data**](#toc1_3_2_)    
    - 1.3.3. [**Preprocess Data**](#toc1_3_3_)    
    - 1.3.4. [**Create Pickle Files**](#toc1_3_4_)    
    - 1.3.5. [**Create Training And Testing Datasets**](#toc1_3_5_)    
    - 1.3.6. [**Build the Model**](#toc1_3_6_)    
    - 1.3.7. [**Predict The Responses**](#toc1_3_7_)    
      - 1.3.7.1. [**Load Required Python Modules**](#toc1_3_7_1_)    
      - 1.3.7.2. [**Establish Environment Variables and Load Data**](#toc1_3_7_2_)    
      - 1.3.7.3. [**Creat Prediction Functions**](#toc1_3_7_3_)    
      - 1.3.7.4. [**Interactive loop for testing**](#toc1_3_7_4_)    

<!-- vscode-jupyter-toc-config
    numbering=true
    anchor=true
    flat=false
    minLevel=1
    maxLevel=6
    /vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

-----------------------------
## 1.1. <a id='toc1_1_'></a>[**Problem Statement**](#toc0_)
-----------------------------

A company holds an event that has been given the deserved promotion through marketing in
hopes of attracting as big an audience as possible. Now, it’s up to the customer support team to
guide the audience and answer any queries. Providing high-quality support and guidance is the
challenge. The chatbot is very helpful for its 24/7 presence and ability to reply instantly.

-----------------------------
## 1.2. <a id='toc1_2_'></a>[**Objectives**](#toc0_)
-----------------------------

Develop a real-time chatbot to engage with the customers in order to boost their
business growth by using NLP and Speech Recognition.

**Domain:** Customer Support

-----------------------------
## 1.3. <a id='toc1_3_'></a>[**Analysis To Be Done**](#toc0_)
-----------------------------

Create a set of prebuilt commands or inputs as a dataset. Here, we use
command .json as Dataset that contains the patterns we need to find and the responses we
want to return to the user.

-----------------------------
### 1.3.1. <a id='toc1_3_1_'></a>[**Ensure Necessary Modules are Installed**](#toc0_)
-----------------------------

In [1]:
%pip install python-dotenv
%pip install nltk
%pip install keras
%pip install SpeechRecognition
%pip install tensorflow
# %pip install pickle - standard library in python does not need to be installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 48.4 MB/s eta 0:00:00


-----------------------------
### 1.3.2. <a id='toc1_3_2_'></a>[**Import Modules and Set Default Environment Variables and Load Data**](#toc0_)
-----------------------------

In [2]:
import json
import pickle
import random
import copy

import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.layers import Input, Activation, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

This section loads essential libraries and initializes the lemmatizer (for processing words) and sets up arrays to store words, classes, and documents. The dataset_dir is defined for file management.

In [5]:
# Download necessary NLTK data if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')

# Directory for dataset and dependency files
dataset_dir = '/content'

# Initialize lists
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents file
with open(f'{dataset_dir}/commands.json') as data_file:
    intents = json.load(data_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


Loads the `commands.json` file, which contains chatbot intents in JSON format. Each intent includes **tags, patterns, and responses**, which will later be used to train the model.

-----------------------------
### 1.3.3. <a id='toc1_3_3_'></a>[**Preprocess Data**](#toc0_)
-----------------------------

In [6]:
# Loop through each intent in the intents dataset
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)

        # Add to documents in our corpus
        documents.append((word_list, intent['tag']))

        # Add to our classes if it's not already there
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize, lower each word, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Display basic stats
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


#### **Explanation**

**Overview**
This part of the script is responsible for preprocessing natural language data from an `intents` dataset to prepare it for training a machine learning model. The script:
1. Tokenizes each sentence pattern in the dataset.
2. Organizes tokenized words and intents into separate lists for easier processing.
3. Lemmatizes and deduplicates words to standardize the vocabulary.
4. Prints basic statistics about the processed data to verify that it is correct and ready for further use.

The output of this section will be three main lists:
- **`words`**: A sorted, unique list of all words in the dataset.
- **`classes`**: A sorted, unique list of intent tags.
- **`documents`**: A list of tuples containing tokenized words and their associated intent tags.

---

**Step-by-Step Explanation of the Code**

**1. Loop Through Each Intent in the Dataset**

This section iterates over each intent in the `intents` dataset, which is structured as a list of intents, each containing `patterns` (example phrases) and a `tag` (the intent category). Here’s a breakdown of each step:

1. **Tokenization**:
   - For each sentence in `patterns`, the `nltk.word_tokenize()` function splits the sentence into individual words, creating a list called `word_list`.
   - This list is then extended into `words`, which will eventually hold every token (word) across all sentences.

2. **Document Creation**:
   - Each sentence and its associated intent tag are combined as a tuple and appended to the `documents` list.
   - This list serves as the core data structure for training, where each entry in `documents` will map a sentence (as a list of words) to a specific intent.

3. **Intent Tag Collection**:
   - The `classes` list is used to collect unique intent tags from the dataset. If an intent tag hasn’t been seen before, it is added to `classes`.
   - This ensures `classes` contains one entry for each unique intent in the dataset.

**2. Lemmatize, Lowercase, and Remove Duplicates from `words`**

This step processes the `words` list to create a standardized vocabulary for the model:

1. **Lemmatization**:
   - The `lemmatizer.lemmatize()` function reduces each word to its base (or root) form, helping to standardize the vocabulary. For example, words like "running" and "runs" are reduced to "run."
   
2. **Lowercasing**:
   - Converts each word to lowercase to ensure that words are treated case-insensitively (i.e., "Hello" and "hello" are treated as the same word).

3. **Duplicate Removal**:
   - The `set()` function removes duplicate words, and the `sorted()` function organizes the vocabulary alphabetically.

4. **Exclusion of Irrelevant Words**:
   - The `ignore_words` list contains words (such as punctuation marks) that are irrelevant to understanding intent. The script excludes any word in `ignore_words` from the final `words` list.

After this step, `words` becomes a sorted list of unique, standardized vocabulary items, which will be used as input features for training.

**3. Sort Classes**

Here, the script sorts and deduplicates the `classes` list, ensuring each intent tag appears only once and in alphabetical order. This list will be the output labels for the model.

**4. Display Basic Statistics**

The script provides a summary of the processed data, including:
- **Number of Documents**: The total count of `(word_list, intent)` pairs in `documents`, representing all examples in the dataset.
- **Number of Classes**: The count of unique intent categories (i.e., classes) in the dataset.
- **Vocabulary Size**: The count of unique lemmatized words in `words`, giving an idea of the feature space size for the model.

This information is useful for confirming that the data was processed as expected and gives insights into the dataset’s composition.

---

**Summary**
This portion of the script tokenizes, lemmatizes, and organizes the text data from the intents dataset. By creating lists of vocabulary words (`words`), unique intent tags (`classes`), and document pairs (`documents`), this script prepares the raw text for further processing and model training. The lemmatization and deduplication ensure that the vocabulary is concise, while the organization into `documents` and `classes` enables straightforward mapping of inputs to outputs in a neural network model.

### 1.3.4. <a id='toc1_3_4_'></a>[**Create Pickle Files**](#toc0_)

In [7]:
pickle.dump(words,open(f'{dataset_dir}/words.pkl','wb'))
pickle.dump(classes,open(f'{dataset_dir}/classes.pkl','wb'))

The processed `words` and `classes` lists are saved as `.pkl` files for later use, making future model training or testing easier without reprocessing.

### 1.3.5. <a id='toc1_3_5_'></a>[**Create Training And Testing Datasets**](#toc0_)

In [8]:
# Define output_empty based on the number of classes
output_empty = [0] * len(classes)

# Create separate lists for training data inputs (X) and outputs (Y)
train_x = []
train_y = []

# Create bag of words for each sentence and corresponding output
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word to create the base form
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create the bag of words array with 1 if word match found in current pattern
    bag = [1 if w in pattern_words else 0 for w in words]

    # Output is '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Append the bag of words and output row to their respective lists
    train_x.append(bag)
    train_y.append(output_row)

# Initial deep verification of train_x and train_y
def verify_array(array, name):
    print(f"Verifying {name}...")
    for i, row in enumerate(array):
        if not isinstance(row, np.ndarray):
            print(f"Row {i} in {name} is not a numpy array. Found type: {type(row)}")
        elif row.shape[0] != array.shape[1]:
            print(f"Row {i} in {name} has inconsistent shape. Expected {array.shape[1]}, found {row.shape[0]}")
        elif row.dtype != np.float32:
            print(f"Row {i} in {name} has incorrect dtype. Expected float32, found {row.dtype}")

# Convert train_x and train_y to numpy arrays
train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

# Deep verification before proceeding
verify_array(train_x, "train_x")
verify_array(train_y, "train_y")

# Ensure consistency by stacking rows
try:
    train_x = np.vstack(train_x)
    train_y = np.vstack(train_y)
    print("train_x and train_y successfully stacked.")
except ValueError as e:
    print(f"Error in stacking train_x or train_y: {e}")
    raise

# Print shapes and types for debugging
print("Shape of train_x:", train_x.shape)
print("Shape of train_y:", train_y.shape)
print("Data type of train_x:", train_x.dtype)
print("Data type of train_y:", train_y.dtype)

# Double-check by printing sample values if needed
print(f"Sample values from train_x:\n", train_x[:5])
print(f"Sample values from train_y:\n", train_y[:5])

Verifying train_x...
Verifying train_y...
train_x and train_y successfully stacked.
Shape of train_x: (47, 88)
Shape of train_y: (47, 9)
Data type of train_x: float32
Data type of train_y: float32
Sample values from train_x:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0

#### **Explanation**

**Overview**
This script is part of a data preprocessing pipeline designed for training a neural network model for intent classification in a chatbot. The script processes natural language data into structured arrays that a machine learning model can understand. This includes creating a "bag of words" representation for each sentence, building a one-hot encoded array for intents, and ensuring consistency within the data.

**Step-by-Step Explanation of the Script**

**1. Define `output_empty` Based on the Number of Classes**

The `output_empty` list serves as a template for encoding each intent. It creates a one-hot encoded list with the length equal to the number of unique classes (intents), filled with zeros. For each pattern, only one position in this array will be set to `1`, representing the associated intent.

**2. Create Separate Lists for Training Data Inputs (`train_x`) and Outputs (`train_y`)**

The `train_x` and `train_y` lists store the input and output data that will be used for training the model:
- **`train_x`**: Holds the "bag of words" representation for each pattern (input sentence).
- **`train_y`**: Holds the one-hot encoded class label for each pattern.

**3. Create Bag of Words and One-Hot Encoded Labels for Each Sentence**

This loop iterates through each document in the `documents` list and performs the following steps:

1. **Bag of Words Creation**:
   - Initializes an empty list `bag` to store the presence/absence of each word.
   - Tokenizes and lemmatizes each word in the pattern to reduce variations to their root forms.
   - For each word in the vocabulary (`words`), checks if it appears in the current pattern. If it does, `1` is added to `bag`; otherwise, `0` is added.

2. **One-Hot Encoding of Output**:
   - Creates a copy of `output_empty`.
   - Sets the index corresponding to the pattern’s intent (from `classes`) to `1`, making this vector a one-hot representation of the intent.

3. **Data Appending**:
   - Adds the completed `bag` and `output_row` to `train_x` and `train_y`, respectively.

**4. Verify the Structure and Consistency of `train_x` and `train_y`**

This function checks the contents of each row in `train_x` and `train_y` to ensure they are consistent:
- **Type Check**: Verifies that each row is a numpy array.
- **Shape Consistency**: Ensures each row has the same shape as expected.
- **Data Type Check**: Confirms each row is of type `float32`, which is required for compatibility with TensorFlow.

**5. Convert `train_x` and `train_y` to Numpy Arrays**

This step converts `train_x` and `train_y` into numpy arrays of type `float32`, which ensures compatibility with the neural network model. The `float32` type is particularly important for TensorFlow.

**6. Perform Deep Verification Using `verify_array`**

After converting the data to numpy arrays, the `verify_array` function is called for both `train_x` and `train_y` to confirm that there are no inconsistencies in type, shape, or data type. This step ensures data integrity before proceeding.

**7. Ensure Consistency by Stacking Rows with `np.vstack`**

The `np.vstack` function is used to stack all rows vertically, forming a consistent 2D array for both `train_x` and `train_y`. If the data has irregularities, this function will raise a `ValueError`, which can help in diagnosing any hidden inconsistencies. 

**8. Print Shapes, Data Types, and Sample Values for Debugging**

These print statements provide final checks of `train_x` and `train_y` by displaying:
- **Shapes**: Ensures both arrays have the expected shape.
- **Data Types**: Confirms that both arrays are of type `float32`.
- **Sample Values**: Provides the first few entries for inspection, allowing a final verification that the data looks correct before model training.

---

**Summary**
This section of the script preprocesses data for model training by creating bag-of-words vectors and one-hot encoded labels for each training example. It verifies data consistency using type, shape, and content checks, making sure everything conforms to the structure expected by TensorFlow.

The output of this section will be two numpy arrays, `train_x` and `train_y`, that are ready to be fed into a neural network model for training. This ensures the data is accurately structured and validated, minimizing errors during the training phase.

### 1.3.6. <a id='toc1_3_6_'></a>[**Build the Model**](#toc0_)

In [10]:
# Define model structure
model = Sequential()
model.add(Input(shape=(train_x.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_y.shape[1], activation='softmax'))

# Compile model with updated learning rate parameter
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train and save the model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save(f'{dataset_dir}/chatbot_model.keras', hist)

print("Model created and saved")


Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.0225 - loss: 2.2943
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0875 - loss: 2.1613      
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1442 - loss: 2.1039 
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2066 - loss: 2.0551  
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2533 - loss: 1.9286     
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4563 - loss: 1.7430 
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4178 - loss: 1.6951 
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5705 - loss: 1.5733 
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6280 - loss: 1.4621 
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6161 - loss: 1.3698 
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5657 - loss: 1.3686 
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s

#### **Explanation**

**Overview**
This section of the script defines, compiles, trains, and saves a neural network model for intent classification in a chatbot. The model is built using Keras and is designed to classify user input into predefined intents. By training on the `train_x` and `train_y` data, the model learns to associate different patterns of words with corresponding intents, making it suitable for use in a chatbot or other natural language understanding applications.

**Step-by-Step Explanation of the Code**

**1. Define Model Structure**

This block defines a neural network model with multiple layers, using Keras' Sequential API to stack each layer in a linear progression:

1. **Input Layer**:
   - `Input(shape=(train_x.shape[1],))` specifies the input shape, which corresponds to the number of features in each training sample (the length of each "bag of words" vector in `train_x`).
   - `train_x.shape[1]` gives the number of features (words) the model expects as input.

2. **First Hidden Layer**:
   - `Dense(128, activation='relu')` is a fully connected layer with 128 neurons and a ReLU activation function.
   - The ReLU activation function introduces non-linearity, allowing the network to learn more complex patterns.

3. **First Dropout Layer**:
   - `Dropout(0.5)` is a dropout layer that randomly sets 50% of the input units to zero during training. This helps prevent overfitting by ensuring the model doesn’t rely too heavily on specific neurons.

4. **Second Hidden Layer**:
   - `Dense(64, activation='relu')` adds a second dense layer with 64 neurons and ReLU activation. This layer refines the learned features from the previous layer.

5. **Second Dropout Layer**:
   - `Dropout(0.5)` applies dropout again with a 50% dropout rate, further reducing overfitting by encouraging the model to generalize.

6. **Output Layer**:
   - `Dense(train_y.shape[1], activation='softmax')` is the output layer, where the number of neurons matches the number of unique classes (intents) in `train_y`.
   - The `softmax` activation function outputs a probability distribution across the classes, making it suitable for multi-class classification. The model will output the probability of each intent, allowing the chatbot to select the most likely intent based on user input.

**2. Compile the Model**

In this step, the model is configured with an optimizer, loss function, and evaluation metric:

1. **Optimizer**:
   - The optimizer used is `SGD` (Stochastic Gradient Descent) with a learning rate of 0.01, momentum of 0.9, and Nesterov momentum enabled.
   - **Momentum** helps accelerate the optimizer in the direction of the gradient, allowing it to navigate more effectively through shallow regions of the loss surface.
   - **Nesterov Momentum** is a variation that looks ahead in the gradient direction, often resulting in faster convergence.

2. **Loss Function**:
   - `categorical_crossentropy` is used as the loss function because this is a multi-class classification problem. Categorical cross-entropy compares the predicted probability distribution of intents with the actual one-hot encoded distribution, guiding the model to adjust weights for better predictions.

3. **Metrics**:
   - `metrics=['accuracy']` tells Keras to calculate and display accuracy during training, allowing us to monitor how well the model is performing on the training data.

**3. Train the Model**

This line initiates the training process. The `fit()` function iteratively updates the model weights to minimize the loss on the training data:

1. **Inputs (`train_x` and `train_y`)**:
   - `train_x` contains the input patterns in the form of bag-of-words vectors.
   - `train_y` contains the one-hot encoded labels for each intent.

2. **Epochs**:
   - `epochs=200` specifies that the model should go through the entire training dataset 200 times, giving it ample opportunity to learn the relationship between input patterns and intents.

3. **Batch Size**:
   - `batch_size=5` splits the training data into mini-batches of 5 samples. The model updates its weights after each batch, which can make training faster and reduce memory usage.

4. **Verbosity**:
   - `verbose=1` enables detailed logging of the training progress, allowing us to see the accuracy and loss values after each epoch.

**4. Save the Model**

After training, the model is saved to disk using `model.save()`. This allows the trained model to be loaded and used later for predictions without retraining. The model file is saved with the `.keras` extension, which stores the model architecture, weights, and optimizer state.

- **File Path**:
  - `f'{dataset_dir}/chatbot_model.keras'` saves the model in the specified directory with a filename of `chatbot_model.keras`.
  
**5. Print Confirmation**

This line confirms that the model has been successfully created and saved, signaling the end of the training process.

---

**Summary**
This section of the script builds, compiles, trains, and saves a neural network model for classifying intents in a chatbot. The model uses two hidden layers with dropout for regularization, and it outputs a probability distribution across possible intents. Training is performed over 200 epochs with mini-batches of size 5. The final model is saved to disk, making it ready for use in a chatbot application. 

This model structure and training configuration enable the chatbot to understand a variety of intents from user input and respond appropriately.

### 1.3.7. <a id='toc1_3_7_'></a>[**Predict The Responses**](#toc0_)

#### 1.3.7.1. <a id='toc1_3_7_1_'></a>[**Load Required Python Modules**](#toc0_)

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np

from tensorflow.keras.models import load_model

#### 1.3.7.2. <a id='toc1_3_7_2_'></a>[**Establish Environment Variables and Load Data**](#toc0_)

In [12]:
# Define dataset directory
dataset_dir = '/content'

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load the trained model
model = load_model(f'{dataset_dir}/chatbot_model.keras')

# Load words and classes
with open(f'{dataset_dir}/words.pkl', 'rb') as f:
    words = pickle.load(f)
with open(f'{dataset_dir}/classes.pkl', 'rb') as f:
    classes = pickle.load(f)

# Load intents
with open(f'{dataset_dir}/commands.json', 'r') as f:
    intents = json.load(f)

#### 1.3.7.3. <a id='toc1_3_7_3_'></a>[**Creat Prediction Functions**](#toc0_)

In [15]:
# Function to clean up the sentence
def clean_up_sentence(sentence):
    # Tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence)
    # Lemmatize each word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to create a bag of words from the sentence
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f'found in bag: {w}')
    return np.array(bag)

# Function to predict the class
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Function to get the response
def get_response(intents_list, intents_json):
    """
    Retrieves the chatbot's response based on predicted intent.

    Args:
        intents_list: A list of predicted intents.
        intents_json: The JSON data containing intents and responses.

    Returns:
        The chatbot's response string.
    """
    if intents_list:
        tag = intents_list[0]['intent']
        # Access the 'intents' key from the intents_json dictionary
        # Previously: list_of_intents = intents_json['intents'] # intents_json is actually a list and not a dictionary
        list_of_intents = intents_json.get('intents', []) # Get the 'intents' key from the dictionary 'intents' declared on line 12 of ipython-input-12-858ce5ee1d16
                                                    # or assign an empty list in case of absence of an 'intents' key.
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
    else:
        result = "I don't understand, please try again."
    return result

# Function to chat with the model
def chatbot_response(text):
    """
    Gets the chatbot's response to the user's input.

    Args:
        text: The user's input text.

    Returns:
        The chatbot's response string.
    """
    intents_list = predict_class(text, model)
    # Pass the intents data (intents) loaded from JSON as the second argument
    # Previously: response = get_response(intents, intents)
    response = get_response(intents_list, intents) # Pass 'intents' variable from line 12 of ipython-input-12-858ce5ee1d16 to get_response() method.
    return response

#### **Explanation**

**Overview**
This script defines functions to process user input and generate a chatbot response based on a trained machine learning model. It includes functions to clean and tokenize input, convert text to a bag-of-words format, predict the most likely intent class, and retrieve a suitable response for the user. The script uses a trained model to classify intents and a JSON file containing the intent definitions and possible responses.

**Step-by-Step Explanation of Each Function**

**1. Function to Clean Up the Sentence**

The `clean_up_sentence` function preprocesses user input to make it consistent with the training data. This ensures that the model receives clean, standardized text, helping to improve prediction accuracy.

1. **Tokenization**: 
   - Uses `nltk.word_tokenize(sentence)` to split the sentence into individual words (tokens).
   
2. **Lemmatization**:
   - Converts each word to lowercase and applies lemmatization using `lemmatizer.lemmatize(word.lower())`, which reduces words to their base form. This helps handle variations of the same word, like "running" and "run," treating them as identical.

The function returns a list of lemmatized words (tokens) from the input sentence.

**2. Function to Create a Bag of Words from the Sentence**

The `bow` (bag-of-words) function converts the input sentence into a numerical format that the model can understand.

1. **Clean and Tokenize**:
   - Calls `clean_up_sentence(sentence)` to preprocess the sentence and convert it into a list of lemmatized words.

2. **Initialize Bag of Words**:
   - Creates an array, `bag`, initialized with zeros. The length of `bag` matches the length of `words` (vocabulary).

3. **Map Words to Bag of Words**:
   - For each word in `sentence_words`, the function checks if it exists in the vocabulary (`words`). If it does, the corresponding index in `bag` is set to `1`.
   - If `show_details` is set to `True`, the function prints each word that is found in `words`.

The function returns a numpy array (`bag`) representing the presence or absence of each word from `words` in the input sentence.

**3. Function to Predict the Class**

The `predict_class` function uses the trained model to predict the most likely intent class for a given sentence.

1. **Convert Sentence to Bag of Words**:
   - Calls `bow(sentence, words, show_details=False)` to convert the sentence into the bag-of-words format.

2. **Make Prediction**:
   - Passes the bag-of-words vector to `model.predict(np.array([p]))[0]` to get prediction probabilities for each class.

3. **Filter Predictions by Error Threshold**:
   - Sets an `ERROR_THRESHOLD` to filter out low-confidence predictions. Only classes with prediction probabilities above `0.25` are kept.

4. **Sort and Prepare Results**:
   - Sorts the results in descending order based on confidence scores.
   - Creates `return_list`, which stores dictionaries with "intent" and "probability" keys for each high-confidence prediction.

The function returns `return_list`, a sorted list of possible intents with their respective confidence probabilities.

**4. Function to Get the Response**

The `get_response` function retrieves an appropriate response from `intents_json` based on the predicted intent.

1. **Identify Top Intent**:
   - Extracts the intent tag with the highest probability from `intents_list` (i.e., the first item in the list).

2. **Search for Responses in JSON**:
   - Searches for the matching intent in `intents_json['intents']`.
   - If a match is found, the function randomly selects a response from `responses` associated with that intent.

3. **Default Response**:
   - If `intents_list` is empty, or no intent matches are found, a default message ("I don't understand, please try again.") is returned.

The function returns the chatbot's response as a string.

**5. Function to Chat with the Model**

The `chatbot_response` function acts as the main interface for interacting with the chatbot. It handles user input, predicts the intent, and retrieves a response.

1. **Predict Class**:
   - Calls `predict_class(text, model)` to determine the most probable intent classes for the input `text`.

2. **Get Response**:
   - Calls `get_response(intents_list, intents)` to fetch a response based on the predicted intents.

The function returns the chatbot’s response as a string.

---

**Summary**
This script processes user input, predicts the intent using a trained model, and returns a response from a predefined JSON file of intents and responses. The sequence of functions is as follows:
1. **`clean_up_sentence`**: Tokenizes and lemmatizes user input.
2. **`bow`**: Converts the sentence into a bag-of-words vector.
3. **`predict_class`**: Predicts the intent class of the input using the model.
4. **`get_response`**: Retrieves a response from the intents JSON file based on the predicted intent.
5. **`chatbot_response`**: Coordinates these functions to provide a complete response to user input.

This modular approach allows each function to perform a specific task, making the code easy to understand and maintain. The final result is a chatbot capable of understanding and responding to user intents based on predefined data and a trained model.


#### 1.3.7.4. <a id='toc1_3_7_4_'></a>[**Interactive loop for testing**](#toc0_)

In [16]:
print("Chatbot is ready to chat! (Type 'exit' to stop)")
while True:
    message = input("You: ")
    if message.lower() == "exit":
        print("Goodbye!")
        break
    response = chatbot_response(message)
    print("Bot:", response)

Chatbot is ready to chat! (Type 'exit' to stop)
You: Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: Hi there, how can I help?
You: Hey, is anyone there?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Hi there, how can I help?
You: Hola
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Hello, thanks for asking
You: Goodbye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: See you!
You: Nice chatting with you, bye!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: Bye! Come back again soon.
You: Till next time
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Bot: See you!
You: Thank you for your help
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: Happy to help!
You: That's helpful
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Any time!
You: Awesome, thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: My pleasure
You: How can you help me?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
You: What support do you offer?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/st

In [18]:
!python --version
print(f"TensorFlow version: {tf.__version__}")

Python 3.10.12
TensorFlow version: 2.17.0
